For reference this page shows what we're trying to do:

https://unidata.github.io/MetPy/latest/examples/cross_section.html

In [1]:
import xarray as xr
import netCDF4
import metpy.calc as mpcalc
from metpy.interpolate import cross_section

In [2]:
ds = xr.open_dataset('nam_cropped.nc')

/home/erick/venv/palisade/lib/python3.12/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


In [3]:
data = ds.metpy.parse_cf().squeeze()

In [4]:
start = (38.0, -121.0)
end = (35.5, -119.0)

Why doesn't this work?

In [5]:
cross = cross_section(data, start, end)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

We can apparently do it as individual variables:

In [6]:
rh1 = cross_section(data.rh, start, end)
tmp1 = cross_section(data.tmp, start, end)
u1 = cross_section(data.u_wind, start, end)
v1 = cross_section(data.v_wind, start, end)

In [7]:
tmp1

<xarray.DataArray 'tmp' (isobaric: 5, index: 100)> Size: 4kB
array([[-7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -7.        ,
        -7.        , -7.        , -7.        , -7.        , -6.87557624,
        -6.59863617, -6.        , -6.        , -6.        , -6.        ,
        -6.        , -6.        , -6.        , -6.        , -6.        ,
        -6.        , -6.        , -6.        , -6.        , -6.        ,
        -6.        , -6.        , -6.        , -6.        , -6.        ,
        -6.        , -6.        , -6.        , -6.        , -6.        ],
...
       [22.        , 22.        , 22.        , 22.14762083, 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.        ,
        22.        , 22.        , 22.        , 22.        , 22.05497018,
        22.41051995, 22.76608156, 22.53773859, 22.47724109, 22.83283938,
        22.44195616, 22.54407403, 22.89971077, 23.        , 22.64879661,
        22.96670096, 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ,
        23.        , 23.        , 23.        , 23.        , 23.        ]])
Coordinates:
  * isobaric   (isobaric) float32 20B 4.75e+04 6e+04 7.25e+04 8.5e+04 9.25e+04
    time       datetime64[ns] 8B 2024-09-30T16:00:00
    metpy_crs  object 8B Projection: lambert_conformal_conic
    x          (index) float64 800B -2.037e+06 -2.036e+06 ... -1.931e+06
    y          (index) float64 800B 2.059e+05 2.027e+05 ... -1.04e+05 -1.071e+05
  * index      (index) int64 800B 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
Attributes:
    units:          celsius
    grid_mapping:   lambert_conformal_conic
    standard_name:  temperature
    long_name:      Temperature

But it won't let us reassemble the resulting variables into a dataset

In [8]:
cross = xr.Dataset(data_vars={'tmp': tmp1, 'rh': rh1, 'u_wind': u1, 'v_wind': v1})

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Some required calculations will work with the individual variables:

In [9]:
ptemp = mpcalc.potential_temperature(tmp1['isobaric'], tmp1)
ptemp

<xarray.DataArray (isobaric: 5, index: 100)> Size: 4kB
<Quantity([[88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.32245223 88.32245223 88.32245223 88.32245223
  88.32245223 88.32245223 88.36374252 88.45564568 88.65430439 88.65430439
  88.65430439 88.65430439 88.65430439 88.65430439 88.65430439 88.65430439
  88.65430439 88.65430439 88.65430439 88.65430439 88.65430439 88.65430439
  88.65430439 88.65430439 88.65430439 88.65430439 88.65430439 88.65430439
  88.65430439 88.65430439 88.65430439 88.65430439]
 [86.65515214 86.65515214 86.65515214 86.61982731 86.46372242 86.34871539
  86.34472709 86.34472709 86.35644091 86.6092527  86.65515214 86.65515214
  86.65515214 86.65515214 86.65515214 86.65515214 86.65515214 86.65515214
...
  82.94308888 82.94308888 82.94308888 82.94308888 82.94308888 82.94308888
  82.97715483 83.224109   83.224109   83.224109   83.224109   83.224109
  83.224109   83.224109   83.224109   83.224109  ]
 [80.96326102 80.96326102 80.96326102 81.00375522 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.96326102 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.96326102 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.96326102 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.96326102 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.96326102 80.96326102 80.96326102
  80.96326102 80.96326102 80.96326102 80.97834001 81.07587167 81.17340657
  81.1107693  81.09417409 81.19171906 81.08449501 81.11250719 81.2100627
  81.23757327 81.14123387 81.22843893 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327 81.23757327 81.23757327
  81.23757327 81.23757327 81.23757327 81.23757327]], 'kelvin')>
Coordinates:
  * isobaric   (isobaric) float32 20B 4.75e+04 6e+04 7.25e+04 8.5e+04 9.25e+04
    time       datetime64[ns] 8B 2024-09-30T16:00:00
    metpy_crs  object 8B Projection: lambert_conformal_conic
    x          (index) float64 800B -2.037e+06 -2.036e+06 ... -1.931e+06
    y          (index) float64 800B 2.059e+05 2.027e+05 ... -1.04e+05 -1.071e+05
  * index      (index) int64 800B 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99

But then, some calculations mysteriously fail:

In [10]:
t_wind, n_wind = mpcalc.cross_section_components(u1, v1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()